In [126]:
!pip install nltk gensim


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [127]:
import pandas as pd
import time
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords
from groq import Groq
import os
from dotenv import load_dotenv

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oskarroeske/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ImportError: cannot import name 'strip' from 'gensim.parsing.preprocessing' (/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gensim/parsing/preprocessing.py)

# EDA

In [56]:
df = pd.read_csv("../data/gpt_fake_recommendations.csv")

In [57]:
df.head()

,Company,Short Name,Date,Target Price,Recommendation,Commentary
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth in cost management, c..."
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL is benefiting from investment in research...
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,AAPL's performance in technology has been stea...
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties in shift in consumer ...
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges in expansion into internatio...


In [58]:
df.value_counts("Recommendation")

Recommendation
Buy     434
Sell    428
Hold    374
Name: count, dtype: int64

In [59]:
df.value_counts("Company")

Company
3M Company                           12
Norfolk Southern Corporation         12
Salesforce, Inc.                     12
S&P Global Inc.                      12
Raytheon Technologies Corporation    12
                                     ..
Exelon Corporation                   12
Eli Lilly and Company                12
Dow Inc.                             12
Deere & Company                      12
Zoetis Inc.                          12
Name: count, Length: 103, dtype: int64

In [60]:
df.groupby(["Short Name"]).mean("Target Price")

,Target Price
Short Name,
AAPL,249.782500
ABBV,227.696667
ADBE,251.938333
ALL,280.011667
AMGN,294.452500
...,...
VZ,259.306667
WFC,246.332500
WMT,251.254167


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Company         1236 non-null   object 
 1   Short Name      1236 non-null   object 
 2   Date            1236 non-null   object 
 3   Target Price    1236 non-null   float64
 4   Recommendation  1236 non-null   object 
 5   Commentary      1236 non-null   object 
dtypes: float64(1), object(5)
memory usage: 58.1+ KB


# Removing Stopwords

In [62]:
df["Commentary"] = df["Commentary"].apply(strip_multiple_whitespaces)

In [63]:
# Show list of stopwords
nltk_stopwords = set(stopwords.words('english'))
print(sorted(nltk_stopwords))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some',

In [64]:
# Remove negations from the list
to_remove=['ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'don', "don't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [65]:
for word in to_remove:
    nltk_stopwords.remove(word)

In [66]:
print(sorted(nltk_stopwords))

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'd', 'did', 'do', 'does', 'doing', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'more', 'most', 'my', 'myself', 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'she', "she's", 'should', "should've", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'we', 'were', 'what', 'when', 'where', 'which', 'while', 'who', 'who

In [67]:
# Correct apply version
df["Commentary"] = df["Commentary"].apply(lambda comment: remove_stopwords(comment, stopwords=nltk_stopwords))

In [68]:
print(df)

                 Company Short Name        Date  Target Price Recommendation  \
0             Apple Inc.       AAPL  2023-01-15        121.35           Hold   
1             Apple Inc.       AAPL  2023-02-15        371.59            Buy   
2             Apple Inc.       AAPL  2023-03-15        202.52           Hold   
3             Apple Inc.       AAPL  2023-04-15        355.88           Hold   
4             Apple Inc.       AAPL  2023-05-15        292.45           Sell   
...                  ...        ...         ...           ...            ...   
1231  Exelon Corporation        EXC  2023-08-15        242.37           Sell   
1232  Exelon Corporation        EXC  2023-09-15        248.48            Buy   
1233  Exelon Corporation        EXC  2023-10-15        148.36           Hold   
1234  Exelon Corporation        EXC  2023-11-15        143.41            Buy   
1235  Exelon Corporation        EXC  2023-12-15        172.69            Buy   

                                       

# Add dates for 3 / 6 / 9 / 12 months

In [69]:
# transform data column to actual date
df["Date"] = pd.to_datetime(df["Date"],format='%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Company         1236 non-null   object        
 1   Short Name      1236 non-null   object        
 2   Date            1236 non-null   datetime64[ns]
 3   Target Price    1236 non-null   float64       
 4   Recommendation  1236 non-null   object        
 5   Commentary      1236 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 58.1+ KB


In [70]:
# Apply DateOffset for each row using 'apply' on the DataFrame
df["Date next Day"] = df["Date"].apply(lambda date: date + pd.DateOffset(days=1))
df["Date after 3 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=3))
df["Date after 6 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=6))
df["Date after 9 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=9))
df["Date after 12 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=12))

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Company               1236 non-null   object        
 1   Short Name            1236 non-null   object        
 2   Date                  1236 non-null   datetime64[ns]
 3   Target Price          1236 non-null   float64       
 4   Recommendation        1236 non-null   object        
 5   Commentary            1236 non-null   object        
 6   Date next Day         1236 non-null   datetime64[ns]
 7   Date after 3 Months   1236 non-null   datetime64[ns]
 8   Date after 6 Months   1236 non-null   datetime64[ns]
 9   Date after 9 Months   1236 non-null   datetime64[ns]
 10  Date after 12 Months  1236 non-null   datetime64[ns]
dtypes: datetime64[ns](6), float64(1), object(4)
memory usage: 106.3+ KB


# API Connection

In [72]:
"""import yfinance as yf
from datetime import timedelta
import pandas as pd
import numpy as np  # Import numpy to use np.abs


#this function might not be necessary since it makes more sense to look at the whole time frame to see if a target price was reached or not!

def get_stock_price_nearest(ticker, date):
    try:
        # Ensure the date is in Pandas Timestamp format
        target_date = pd.to_datetime(date)

        # Define a range around the target date to fetch data
        start_date = target_date - timedelta(days=5)
        end_date = target_date + timedelta(days=5)

        # Download data within this range
        stock_data = yf.download(ticker, start=start_date, end=end_date)

        if not stock_data.empty:
            # Check if data exists for the exact date
            if target_date in stock_data.index:
                return stock_data.loc[target_date]['Close']
            else:
                # Manually find the closest available trading day using numpy.abs()
                stock_data['diff'] = np.abs(stock_data.index - target_date)
                closest_date = stock_data['diff'].idxmin()  # Get the index of the closest date
                closest_price = stock_data.loc[closest_date]['Close']
                return closest_price
        else:
            return None
    
    except Exception as e:
        return f"An unexpected error occurred for {ticker}: {str(e)}"

#def get_max_stock_price_for_timeframe(short_name,time_frame):
    """

'import yfinance as yf\nfrom datetime import timedelta\nimport pandas as pd\nimport numpy as np  # Import numpy to use np.abs\n\n\n#this function might not be necessary since it makes more sense to look at the whole time frame to see if a target price was reached or not!\n\ndef get_stock_price_nearest(ticker, date):\n    try:\n        # Ensure the date is in Pandas Timestamp format\n        target_date = pd.to_datetime(date)\n\n        # Define a range around the target date to fetch data\n        start_date = target_date - timedelta(days=5)\n        end_date = target_date + timedelta(days=5)\n\n        # Download data within this range\n        stock_data = yf.download(ticker, start=start_date, end=end_date)\n\n        if not stock_data.empty:\n            # Check if data exists for the exact date\n            if target_date in stock_data.index:\n                return stock_data.loc[target_date][\'Close\']\n            else:\n                # Manually find the closest available trad

In [129]:
import pandas as pd

# Load and ensure 'Date' is in datetime format
df_performance_data = pd.read_csv("../data/performance_data.csv")

# Convert 'Date' column to datetime and set it as index (with timezone awareness)
df_performance_data['Date'] = pd.to_datetime(df_performance_data['Date'], utc=True)
df_performance_data = df_performance_data.set_index('Date')

# Function to get the maximum stock price within a date range or the nearest available date
def get_stock_prices(ticker, start_date, end_date=None):
    try:
        # Ensure start_date and end_date are in the same timezone (UTC)
        start_date = pd.to_datetime(start_date, utc=True)
        if end_date is not None:
            end_date = pd.to_datetime(end_date, utc=True)
        else:
            end_date = start_date  # If no end_date, use start_date for a single day

        # Filter the data for the ticker and date range
        filtered_data = df_performance_data.loc[
            (df_performance_data.index >= start_date) & 
            (df_performance_data.index <= end_date), ticker]

        if not filtered_data.empty:
            # Return the maximum price within the filtered date range
            max_price = filtered_data.max()
            min_price = filtered_data.min()
            return float(max_price),float(min_price)
        else:
            # If no data available in the range, find the next available date using asof()
            next_available_data = df_performance_data[ticker].asof(start_date)
            if next_available_data is not None:
                return float(next_available_data)
            else:
                return float('nan'),float('nan')  # Return NaN for missing data
    
    except KeyError:
        return float('nan'),float('nan')  # Return NaN for missing data
    except Exception as e:
        return float('nan'),float('nan')  # Return NaN for missing data

# Example function call
print(get_stock_prices(df["Short Name"][0], df["Date"][0], df["Date next Day"][0]))


133.50433349609375


In [130]:
print(get_stock_prices(df["Short Name"][0], df["Date"][0], df["Date after 3 Months"][0]))

(164.8730010986328, 133.9501495361328)


In [135]:
def calculate_prices(row):
    short_name = row['Short Name']
    row['Start Price'] = get_stock_prices(short_name, row['Date'])
    row['One Day after'] = get_stock_prices(short_name,row['Date next Day'])
    row['Max Price after 3 Months'],row["Min Price after 3 Months"] = get_stock_prices(short_name, row['Date'],row['Date after 3 Months'])
    row['Max Price after 6 Months'],row['Min Price after 6 Months'] = get_stock_prices(short_name,row['Date after 3 Months'], row['Date after 6 Months'])
    row['Max Price after 9 Months'],row['Min Price after 9 Months'] = get_stock_prices(short_name, row['Date after 6 Months'], row['Date after 9 Months'])
    #row['Price after 12 Months'] = get_stock_prices(short_name, row['Date after 12 Months'])
    return row
# Apply the function to each row
df = df.apply(calculate_prices, axis=1)

In [136]:
df

,Company,Short Name,Date,Target Price,Recommendation,Commentary,Date next Day,Date after 3 Months,Date after 6 Months,Date after 9 Months,...,tp_reached_after_3_months,tp_reached_after_6_months,tp_reached_after_9_months,provided_arguments,Max Price after 3 Months,Min Price after 3 Months,Min Price after 6 Months,Min Price after 9 Months,Max Price after 6 Months,Max Price after 9 Months
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth cost management, conc...",2023-01-16,2023-04-15,2023-07-15,2023-10-15,...,True,True,True,"{""Cost Management"": ""positive"", ""Technological...",164.873001,133.950150,162.481842,169.562317,192.722244,195.186279
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL benefiting investment research developmen...,2023-02-16,2023-05-15,2023-08-15,2023-11-15,...,False,False,False,"{""Research and Development"": ""positive"", ""Grow...",172.393860,144.175842,170.456406,166.040359,195.186279,188.734222
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,"AAPL's performance technology steady, risks su...",2023-03-16,2023-06-15,2023-09-15,2023-12-15,...,False,False,False,"{""Performance"": ""neutral"", ""Technology"": ""posi...",184.813461,151.795898,173.114151,166.040359,195.186279,197.361084
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties shift consumer prefer...,2023-04-16,2023-07-15,2023-10-15,2024-01-15,...,False,False,False,"{""Consumer Preferences"": ""negative"", ""Demand"":...",192.722244,162.481842,169.562317,166.040359,195.186279,197.361084
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges expansion international mark...,2023-05-16,2023-08-15,2023-11-15,2024-02-15,...,False,False,False,"{""International Expansion"": ""negative"", ""Techn...",195.186279,170.456406,166.040359,180.495071,188.734222,197.361084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Exelon Corporation,EXC,2023-08-15,242.37,Sell,With ongoing issues like regulatory pressure e...,2023-08-16,2023-11-15,2024-02-15,2024-05-15,...,False,False,False,"{""Regulatory Pressure"": ""negative"", ""Environme...",40.194012,35.169762,32.599857,33.676178,39.755924,38.337326
1232,Exelon Corporation,EXC,2023-09-15,248.48,Buy,EXC's continued growth environmental regulatio...,2023-09-16,2023-12-15,2024-03-15,2024-06-15,...,False,False,False,"{""Growth Prospects"": ""positive"", ""Regulatory E...",40.011482,34.413116,32.599857,34.397678,36.447979,38.337326
1233,Exelon Corporation,EXC,2023-10-15,148.36,Hold,EXC faces uncertainties shift consumer prefere...,2023-10-16,2024-01-15,2024-04-15,2024-07-15,...,False,False,False,"{""Market Volatility"": ""negative"", ""Consumer Pr...",39.755924,33.404675,32.599857,33.892849,37.114410,38.337326
1234,Exelon Corporation,EXC,2023-11-15,143.41,Buy,EXC benefiting labor shortages strong demand. ...,2023-11-16,2024-02-15,2024-05-15,2024-08-15,...,False,False,False,"{""Labor Market"": ""positive"", ""Demand"": ""positi...",39.755924,32.599857,33.676178,33.892849,38.337326,38.376923


In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Company                    1236 non-null   object        
 1   Short Name                 1236 non-null   object        
 2   Date                       1236 non-null   datetime64[ns]
 3   Target Price               1236 non-null   float64       
 4   Recommendation             1236 non-null   object        
 5   Commentary                 1236 non-null   object        
 6   Date next Day              1236 non-null   datetime64[ns]
 7   Date after 3 Months        1236 non-null   datetime64[ns]
 8   Date after 6 Months        1236 non-null   datetime64[ns]
 9   Date after 9 Months        1236 non-null   datetime64[ns]
 10  Date after 12 Months       1236 non-null   datetime64[ns]
 11  Start Price                1236 non-null   object        
 12  One Da

# Calculate performance of Target Prices

In [138]:
#If the target Price is below the current price, we need to check if it was below the target price
def classifcy_performance(df):
    if df["Target Price"] > df["Start Price"]:
        df["tp_reached_after_3_months"] = df["Target Price"] <= df["Max Price after 3 Months"]
        df["tp_reached_after_6_months"] = df["Target Price"] <= df["MaxPrice after 6 Months"]
        df["tp_reached_after_9_months"] = df["Target Price"] <= df["Max Price after 9 Months"]
    else:
        df["tp_reached_after_3_months"] = df["Target Price"] >= df["Min Price after 3 Months"]
        df["tp_reached_after_6_months"] = df["Target Price"] >= df["Min Price after 6 Months"]
        df["tp_reached_after_9_months"] = df["Target Price"] >= df["Min Price after 9 Months"]

In [139]:
df

,Company,Short Name,Date,Target Price,Recommendation,Commentary,Date next Day,Date after 3 Months,Date after 6 Months,Date after 9 Months,...,tp_reached_after_3_months,tp_reached_after_6_months,tp_reached_after_9_months,provided_arguments,Max Price after 3 Months,Min Price after 3 Months,Min Price after 6 Months,Min Price after 9 Months,Max Price after 6 Months,Max Price after 9 Months
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth cost management, conc...",2023-01-16,2023-04-15,2023-07-15,2023-10-15,...,True,True,True,"{""Cost Management"": ""positive"", ""Technological...",164.873001,133.950150,162.481842,169.562317,192.722244,195.186279
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL benefiting investment research developmen...,2023-02-16,2023-05-15,2023-08-15,2023-11-15,...,False,False,False,"{""Research and Development"": ""positive"", ""Grow...",172.393860,144.175842,170.456406,166.040359,195.186279,188.734222
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,"AAPL's performance technology steady, risks su...",2023-03-16,2023-06-15,2023-09-15,2023-12-15,...,False,False,False,"{""Performance"": ""neutral"", ""Technology"": ""posi...",184.813461,151.795898,173.114151,166.040359,195.186279,197.361084
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties shift consumer prefer...,2023-04-16,2023-07-15,2023-10-15,2024-01-15,...,False,False,False,"{""Consumer Preferences"": ""negative"", ""Demand"":...",192.722244,162.481842,169.562317,166.040359,195.186279,197.361084
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges expansion international mark...,2023-05-16,2023-08-15,2023-11-15,2024-02-15,...,False,False,False,"{""International Expansion"": ""negative"", ""Techn...",195.186279,170.456406,166.040359,180.495071,188.734222,197.361084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Exelon Corporation,EXC,2023-08-15,242.37,Sell,With ongoing issues like regulatory pressure e...,2023-08-16,2023-11-15,2024-02-15,2024-05-15,...,False,False,False,"{""Regulatory Pressure"": ""negative"", ""Environme...",40.194012,35.169762,32.599857,33.676178,39.755924,38.337326
1232,Exelon Corporation,EXC,2023-09-15,248.48,Buy,EXC's continued growth environmental regulatio...,2023-09-16,2023-12-15,2024-03-15,2024-06-15,...,False,False,False,"{""Growth Prospects"": ""positive"", ""Regulatory E...",40.011482,34.413116,32.599857,34.397678,36.447979,38.337326
1233,Exelon Corporation,EXC,2023-10-15,148.36,Hold,EXC faces uncertainties shift consumer prefere...,2023-10-16,2024-01-15,2024-04-15,2024-07-15,...,False,False,False,"{""Market Volatility"": ""negative"", ""Consumer Pr...",39.755924,33.404675,32.599857,33.892849,37.114410,38.337326
1234,Exelon Corporation,EXC,2023-11-15,143.41,Buy,EXC benefiting labor shortages strong demand. ...,2023-11-16,2024-02-15,2024-05-15,2024-08-15,...,False,False,False,"{""Labor Market"": ""positive"", ""Demand"": ""positi...",39.755924,32.599857,33.676178,33.892849,38.337326,38.376923


# Topic Modeling with LDA

# FINBERT

# KeyBERT

# LLM Argument Mining

- Provide List of pre-defined arguments

In [116]:
def find_arguments(text, api_key):
    client = Groq(api_key=api_key)

    instructions = """I will give you a financial text, and here are the instructions:
-Collect the main arguments and assign them to a category.
-Categories must be short, generic, and a maximum of 3 words.
-Check if the argument fits any of the existing categories.
-Create a new category if no match is found.
-Update the existing categories afterwards
-Ignore Target Prices/Recommendation as arguments.
-Classify arguments as positive/negative/neutral based on wording, but only return the raw categories, excluding any words that suggest sentiment (e.g., 'Cost Management' instead of 'Cost Management Growth').
-Return the arguments and sentiment as a dictionary (e.g., {'Category 1': positive, 'Category 2': negative,...).
-the result should always be in one line and every key and value should be within quotation marks
-Return only the dictionary, no other text."""
    final_prompt = instructions + "\n Text: " +text

    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Response must be correct formatted json. No additional text must be provided.",
            },
            {
                "role": "user",
                "content": final_prompt,
            }
        ],
        temperature=0,
        model="llama3-70b-8192",
    )

    return response.choices[0].message.content

In [117]:
# Define a helper function that adds a delay
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

def find_arguments_with_delay(commentary):
    result = find_arguments(commentary,api_key)
    print(result)
    time.sleep(2.5)  # 2.5-second delay
    return result


In [118]:
df["provided_arguments"] = df["Commentary"].apply(find_arguments_with_delay)

{"Cost Management": "positive", "Technological Innovation": "negative", "Company Performance": "neutral"}
{"Research and Development": "positive", "Growth Prospects": "positive", "Technology": "positive"}
{"Performance": "neutral", "Technology": "positive", "Risks": "negative", "International Markets": "negative"}
{"Consumer Preferences": "negative", "Demand": "positive", "Technology": "positive", "Outlook": "negative"}
{"International Expansion": "negative", "Technological Innovation": "positive"}
{"Growth Prospects": "positive", "Regulatory Pressures": "negative", "Geopolitical Concerns": "negative", "Technology Performance": "positive"}
{"Digital Services": "positive", "Competition": "negative", "Innovation": "positive", "Labor Shortages": "negative"}
{"Consumer Preferences": "negative", "Technology Space": "negative"}
{"Market Opportunities": "positive", "Research and Development": "negative", "Investment": "neutral"}
{"Cost Management": "positive", "Geopolitical Risks": "negative"

In [120]:
df

,Company,Short Name,Date,Target Price,Recommendation,Commentary,Date next Day,Date after 3 Months,Date after 6 Months,Date after 9 Months,Date after 12 Months,Start Price,One Day after,Price after 3 Months,Price after 6 Months,Price after 9 Months,tp_reached_after_3_months,tp_reached_after_6_months,tp_reached_after_9_months,provided_arguments
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth cost management, conc...",2023-01-16,2023-04-15,2023-07-15,2023-10-15,2024-01-15,133.504333,133.504333,164.873001,192.722244,195.186279,True,True,True,"{""Cost Management"": ""positive"", ""Technological..."
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL benefiting investment research developmen...,2023-02-16,2023-05-15,2023-08-15,2023-11-15,2024-02-15,154.117630,152.510254,172.393860,195.186279,188.734222,False,False,False,"{""Research and Development"": ""positive"", ""Grow..."
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,"AAPL's performance technology steady, risks su...",2023-03-16,2023-06-15,2023-09-15,2023-12-15,2024-03-15,151.795898,154.633575,184.813461,195.186279,197.361084,False,False,False,"{""Performance"": ""neutral"", ""Technology"": ""posi..."
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties shift consumer prefer...,2023-04-16,2023-07-15,2023-10-15,2024-01-15,2024-04-15,163.920502,163.920502,192.722244,195.186279,197.361084,False,False,False,"{""Consumer Preferences"": ""negative"", ""Demand"":..."
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges expansion international mark...,2023-05-16,2023-08-15,2023-11-15,2024-02-15,2024-05-15,170.963104,170.963104,195.186279,188.734222,197.361084,False,False,False,"{""International Expansion"": ""negative"", ""Techn..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Exelon Corporation,EXC,2023-08-15,242.37,Sell,With ongoing issues like regulatory pressure e...,2023-08-16,2023-11-15,2024-02-15,2024-05-15,2024-08-15,38.032528,38.243870,40.194012,39.755924,38.337326,False,False,False,"{""Regulatory Pressure"": ""negative"", ""Environme..."
1232,Exelon Corporation,EXC,2023-09-15,248.48,Buy,EXC's continued growth environmental regulatio...,2023-09-16,2023-12-15,2024-03-15,2024-06-15,2024-09-15,40.011482,40.011482,40.011482,36.447979,38.337326,False,False,False,"{""Growth Prospects"": ""positive"", ""Regulatory E..."
1233,Exelon Corporation,EXC,2023-10-15,148.36,Hold,EXC faces uncertainties shift consumer prefere...,2023-10-16,2024-01-15,2024-04-15,2024-07-15,2024-10-15,38.378361,38.589710,39.755924,37.114410,38.337326,False,False,False,"{""Market Volatility"": ""negative"", ""Consumer Pr..."
1234,Exelon Corporation,EXC,2023-11-15,143.41,Buy,EXC benefiting labor shortages strong demand. ...,2023-11-16,2024-02-15,2024-05-15,2024-08-15,2024-11-15,38.466282,38.282047,39.755924,38.337326,38.376923,False,False,False,"{""Labor Market"": ""positive"", ""Demand"": ""positi..."


In [176]:
df.to_csv("with_dictionaries.csv")

In [94]:
# Remove newline characters and HTML tags
#df["provided_arguments"] = df["provided_arguments"].str.replace(r'(\n|<.*?>)', '', regex=True)

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Company                    1236 non-null   object        
 1   Short Name                 1236 non-null   object        
 2   Date                       1236 non-null   datetime64[ns]
 3   Target Price               1236 non-null   float64       
 4   Recommendation             1236 non-null   object        
 5   Commentary                 1236 non-null   object        
 6   Date next Day              1236 non-null   datetime64[ns]
 7   Date after 3 Months        1236 non-null   datetime64[ns]
 8   Date after 6 Months        1236 non-null   datetime64[ns]
 9   Date after 9 Months        1236 non-null   datetime64[ns]
 10  Date after 12 Months       1236 non-null   datetime64[ns]
 11  Start Price                936 non-null    float64       
 12  One Da

In [146]:
import pandas as pd
import json
from collections import Counter, defaultdict

# Initialize a dictionary to hold counts for each key and sentiment
sentiment_counts = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))

# Loop through each entry in the provided_arguments column
for entry in df["provided_arguments"]:
    # Ensure the entry is a dictionary
    if isinstance(entry, str):
        try:
            # Convert string to dictionary if it's in JSON-like format
            arguments_dict = json.loads(entry.replace("'", '"'))  # Replace single quotes with double quotes for JSON
        except json.JSONDecodeError:
            continue  # Skip any rows that can't be parsed as dictionaries
    elif isinstance(entry, dict):
        arguments_dict = entry
    else:
        continue  # Skip if entry is not a dictionary or parseable string

    # Update counts for each key and sentiment
    for key, sentiment in arguments_dict.items():
        sentiment_counts[key][sentiment] += 1

# Convert the result to a DataFrame for easier viewing
sentiment_overview_df = pd.DataFrame(sentiment_counts).T
sentiment_overview_df.columns = ["positive", "neutral", "negative"]
sentiment_overview_df.index.name = "Argument"

# Display the overview DataFrame
sentiment_overview_df.reset_index(inplace=True)
sentiment_overview_df.sort_values(by=["positive","negative","neutral"],ascending=False)


,Argument,positive,neutral,negative
6,Performance,117,49,2
21,Market Opportunities,93,25,23
10,Demand,88,0,16
56,Growth,82,0,26
16,Digital Services,70,25,35
...,...,...,...,...
251,Restaurant Outlook,0,1,0
262,Real Estate Performance,0,1,0
278,Brokerage Performance,0,1,0
299,Supplier Management,0,1,0


# Check amount of intensifiers

In [150]:

# Basic Intensifiers
basic_intensifiers = [
    "very", "really", "quite", "so", "too", "totally", "absolutely",
    "completely", "utterly", "entirely", "truly", "fully", "genuinely", "surely"
]

# Strong Intensifiers (includes previous stronger, superlative, and "all/full" intensifiers)
strong_intensifiers = [
    "extremely", "incredibly", "highly", "immensely", "vastly", "remarkably", 
    "tremendously", "phenomenally", "exceptionally", "noticeably", "supremely", 
    "powerfully", "acutely", "enormously", "greatly", "immeasurably", "infinitely",
    "wildly", "lavishly", "ludicrously", "fiercely", "ferociously", "intensely",
    "extraordinarily", "overwhelmingly", "staggeringly", "excessively", "insanely", 
    "unbelievably", "inconceivably", "ridiculously", "crazily", "all-out", 
    "all-around", "full-on", "fully", "flat-out"
]

# Formal/Emotive Intensifiers (combines emotive and formal language for a more academic or dramatic tone)
formal_emotive_intensifiers = [
    "profoundly", "considerably", "significantly", "markedly", "appreciably", 
    "notably", "exceedingly", "far", "deeply", "awfully", "horribly", "dreadfully", 
    "frightfully", "painfully", "absurdly", "scandalously", "outrageously", 
    "shockingly", "horrendously", "terribly", "hideously", "grossly"
]


In [171]:
import re

# Function to count intensifiers in a single text
def count_intensifiers(text, intensifiers):
    # Convert to lowercase to make it case-insensitive
    words = re.findall(r'\b\w+\b', text.lower())
    return sum(words.count(word) for word in intensifiers)

In [173]:
print(count_intensifiers( "This is a very very and extremely argument", intensifiers=basic_intensifiers))

2


In [174]:
# Apply the function to the "Commentary" column of the DataFrame
df["Basic Intensifier"] = df["Commentary"].apply(count_intensifiers, intensifiers=basic_intensifiers)
df["Strong Intensifier"] = df["Commentary"].apply(count_intensifiers, intensifiers=basic_intensifiers)
df["Formal Intensifier"] = df["Commentary"].apply(count_intensifiers, intensifiers=basic_intensifiers)

In [175]:
df

,Company,Short Name,Date,Target Price,Recommendation,Commentary,Date next Day,Date after 3 Months,Date after 6 Months,Date after 9 Months,...,provided_arguments,Max Price after 3 Months,Min Price after 3 Months,Min Price after 6 Months,Min Price after 9 Months,Max Price after 6 Months,Max Price after 9 Months,Basic Intensifier,Strong Intensifier,Formal Intensifier
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth cost management, conc...",2023-01-16,2023-04-15,2023-07-15,2023-10-15,...,"{""Cost Management"": ""positive"", ""Technological...",164.873001,133.950150,162.481842,169.562317,192.722244,195.186279,0,0,0
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL benefiting investment research developmen...,2023-02-16,2023-05-15,2023-08-15,2023-11-15,...,"{""Research and Development"": ""positive"", ""Grow...",172.393860,144.175842,170.456406,166.040359,195.186279,188.734222,0,0,0
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,"AAPL's performance technology steady, risks su...",2023-03-16,2023-06-15,2023-09-15,2023-12-15,...,"{""Performance"": ""neutral"", ""Technology"": ""posi...",184.813461,151.795898,173.114151,166.040359,195.186279,197.361084,0,0,0
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties shift consumer prefer...,2023-04-16,2023-07-15,2023-10-15,2024-01-15,...,"{""Consumer Preferences"": ""negative"", ""Demand"":...",192.722244,162.481842,169.562317,166.040359,195.186279,197.361084,0,0,0
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges expansion international mark...,2023-05-16,2023-08-15,2023-11-15,2024-02-15,...,"{""International Expansion"": ""negative"", ""Techn...",195.186279,170.456406,166.040359,180.495071,188.734222,197.361084,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Exelon Corporation,EXC,2023-08-15,242.37,Sell,With ongoing issues like regulatory pressure e...,2023-08-16,2023-11-15,2024-02-15,2024-05-15,...,"{""Regulatory Pressure"": ""negative"", ""Environme...",40.194012,35.169762,32.599857,33.676178,39.755924,38.337326,0,0,0
1232,Exelon Corporation,EXC,2023-09-15,248.48,Buy,EXC's continued growth environmental regulatio...,2023-09-16,2023-12-15,2024-03-15,2024-06-15,...,"{""Growth Prospects"": ""positive"", ""Regulatory E...",40.011482,34.413116,32.599857,34.397678,36.447979,38.337326,0,0,0
1233,Exelon Corporation,EXC,2023-10-15,148.36,Hold,EXC faces uncertainties shift consumer prefere...,2023-10-16,2024-01-15,2024-04-15,2024-07-15,...,"{""Market Volatility"": ""negative"", ""Consumer Pr...",39.755924,33.404675,32.599857,33.892849,37.114410,38.337326,0,0,0
1234,Exelon Corporation,EXC,2023-11-15,143.41,Buy,EXC benefiting labor shortages strong demand. ...,2023-11-16,2024-02-15,2024-05-15,2024-08-15,...,"{""Labor Market"": ""positive"", ""Demand"": ""positi...",39.755924,32.599857,33.676178,33.892849,38.337326,38.376923,0,0,0


In [155]:
df["Commentary"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1236 entries, 0 to 1235
Series name: Commentary
Non-Null Count  Dtype 
--------------  ----- 
1236 non-null   object
dtypes: object(1)
memory usage: 9.8+ KB


In [164]:
print(df[df["Formal Intensifier"] > 1])

Empty DataFrame
Columns: [Company, Short Name, Date, Target Price, Recommendation, Commentary, Date next Day, Date after 3 Months, Date after 6 Months, Date after 9 Months, Date after 12 Months, Start Price, One Day after, Price after 3 Months, Price after 6 Months, Price after 9 Months, tp_reached_after_3_months, tp_reached_after_6_months, tp_reached_after_9_months, provided_arguments, Max Price after 3 Months, Min Price after 3 Months, Min Price after 6 Months, Min Price after 9 Months, Max Price after 6 Months, Max Price after 9 Months, Basic Intensifier, Strong Intensifier, Formal Intensifier]
Index: []

[0 rows x 29 columns]
